# Visualizing Structures with MDAnalysis and Narupa

In this example, we visualize LSD bound into 5-HT2B receptor with MDAnalysis. We'll explore setting up custom visualizations that will be synchronized with users in VR

## Serving the structure as a Narupa frame

First, we're going to read in the system with MDAnalysis. MDAnalysis has to guess the bonds because there aren't any defined in the PDB file, so it'll take a while.

In [ ]:
import MDAnalysis as mda

In [ ]:
universe = mda.Universe(
    'serotonine_receptor.pdb',
    guess_bonds=True, vdwradii={'Na': 0, 'Cl': 0},
)

Next, we set up a Narupa server that's set up for serving structures, or as we call them, *frames*. 

In [ ]:
from narupa.app import NarupaFrameApplication

In [ ]:
frame_server = NarupaFrameApplication.basic_server(port=0) # Let the OS choose a port for us

In [ ]:
print(f'{frame_server.name}, serving at {frame_server.address}:{frame_server.port}')

Now, let's convert the MDAnalysis universe to the Narupa frame object and send it! We have a handy method for doing just that

In [ ]:
from narupa.mdanalysis import mdanalysis_to_frame_data

In [ ]:
frame = mdanalysis_to_frame_data(universe)

In [ ]:
frame_server.frame_publisher.send_frame(0,frame)

Done! If you connect to the server from VR, you'll see something like this:

![LSD Ball and Stick](images/lsd_ball_and_stick.png)


# Let's make it look good

The ball and stick representation is fine, but we can customize it right here from the notebook. First, we connect a client, which will ask the server to change how things look.

**Note**: In future releases, the following will probably be simplified, and you'll be able to do it from within VR.

In [ ]:
from narupa.app import NarupaImdClient

In [ ]:
client = NarupaImdClient.connect_to_single_server(port=frame_server.port)

We need to join the multiplayer, so we can update settings for everyone else (i.e., your VR app)

In [ ]:
client.subscribe_multiplayer()

The first thing we'll do is hide the *root* layer, and slowly layer things back up.
The `modify()` context allows us to make a bunch of changes, which will all be sent to everyone when we're done

In [ ]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True

Now, let's set up some CPK colours for the main atoms

In [ ]:
cpk_colours = {
    'N': 'blue',
    'P': '#dca523',
    'C': '#c0c0c0',
    'O': '#fc1c03',
    'S': '#e9ce16'
}

We define a utility method that converts from an MDAnalysis selection string to an array of integers that define a selection in Narupa

In [ ]:
def generate_mdanalysis_selection(selection: str, universe):
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

Create a selection of anything that's a protein, and ignore hydrogens

In [ ]:
protein = client.create_selection("Protein", [])
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H", universe))

Next, create a selection of the LSD (and other ligands)

In [ ]:
# Select ligands
ligands = client.create_selection("Ligands", [])
with ligands.modify():
    ligands.set_particles(generate_mdanalysis_selection("not protein and not resname DPPC", universe))

And we happen to know that DPPC refers to the lipids, so we grab that too

In [ ]:
lipids = client.create_selection("Lipids", [])
with lipids.modify():
    lipids.set_particles(generate_mdanalysis_selection("resname DPPC", universe))

We define another useful function, that generates a colour gradients from matplotlib. 

In [ ]:
import matplotlib.cm
def get_matplotlib_gradient(name: str):
    cmap = matplotlib.cm.get_cmap(name)
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

Now, we'll render the protein using our tetrahedral spline renderer, coloured with the lovely viridis colour scheme. 
We do this by editting the `renderer` settings, which is just a dictionary of settings. 

In [ ]:
# Render the protein
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'render': 'geometric spline'
        }
    protein.interaction_method = 'none'

Alternatively, you can colour by secondary structure:

In [ ]:
with protein.modify():
    protein.renderer = 'cartoon'
    protein.interaction_method = 'none'

We'll add the ligands back in with CPK liquorice

In [ ]:
with ligands.modify():
    ligands.renderer = {
            'color': {
                'type': 'cpk',
                'scheme': cpk_colours,
            },
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligands.interaction_method = 'none'

And the lipids with a scaled down liquorice

In [ ]:
with lipids.modify():
    lipids.renderer = {
            'color': {
                'type': 'cpk',
                'scheme': cpk_colours,
            },
            'scale': 0.01,
            'render': 'liquorice'
        }
    lipids.interaction_method = 'none'

You may find the lipids are a bit much - you can easily hide them. Just uncomment out and run these lines

In [ ]:
# with lipids.modify():
#     lipids.hide = True

## Visualization based on distance

Finally, we'll render the sidechains near ligand

In [ ]:
nearby = client.create_selection("Nearby", [])

In [ ]:
with nearby.modify():
    nearby.set_particles(generate_mdanalysis_selection("(protein and (not backbone or name CA)) and same resid as around 4 resname 7LD", universe))

In [ ]:
with nearby.modify():
    nearby.renderer = {
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'scale': 0.05,
            'render': 'cycles'
        }
    nearby.interaction_method = 'none'

In [ ]:
with nearby.modify():
    nearby.renderer = {
            'color': {
                'type': 'cpk',
                'scheme': cpk_colours,
            },
            'scale': 0.05,
            'render': 'liquorice'
        }
    nearby.interaction_method = 'none'

Depending on which cells you ran, you'll have something that looks like this, much better! 

![LSD Narupa](images/lsd_narupa.png)

# Tidying Up

In [ ]:
client.clear_selections()

In [ ]:
client.close()

In [ ]:
frame_server.close()

# Next Steps

* Build your own [VR trajectory viewer](./mdanalysis_trajectory.ipynb) with MDAnalysis and Narupa
* Dig into how [frames](../fundamentals/frame.ipynb) are constructed under the hood. 